In [9]:
import requests
import json
import os
import re
from datetime import datetime
import pandas as pd
from pandas import json_normalize
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [10]:
base_url = os.getenv('os2iot_BASE_URL')
os2iot_url = os.getenv('os2iot_URL')
os2iot_api = os.getenv('os2iot_api')
org_id = os.getenv('os2iot_org_id')
kerlink_password = os.getenv('kerlink_password')

url = f"{base_url}/chirpstack/gateway?organizationId={org_id}&limit=100000&offset=0"

headers = {
  'X-API-KEY': os2iot_api
}

response = requests.get(url, headers=headers)

data = json.loads(response.text)

In [11]:
for item in data['resultList']:
    location = item.get('location', {})
    item['latitude'] = location.get('latitude')
    item['longitude'] = location.get('longitude')

for item in data['resultList']:
    tags = item.get('tags', {})
    item['phone'] = tags.get('Phone')
    item['MAC'] = tags.get('MAC')
    item['IP'] = tags.get('IP')

gateway_df = pd.DataFrame(data.get('resultList', []))[['gatewayId',
                                                       'name',
                                                       'phone',
                                                       #'operationalResponsibleName',
                                                       #'rxPacketsReceived',
                                                       #'txPacketsEmitted', 
                                                       # 'modelName',
                                                       # 'antennaType', 
                                                       'placement', 
                                                       #'latitude',
                                                       #'longitude',
                                                       'lastSeenAt',
                                                       'status']]

gateway_df = gateway_df.rename(columns={'gatewayId': 'gatewayEUI',
                                        'name': 'gateway_name'})

today_str = datetime.today().strftime("%Y-%m-%d")

gateway_df['lastSeenAt'] = pd.to_datetime(gateway_df['lastSeenAt'])
gateway_df['lastSeenAt'] = gateway_df['lastSeenAt'].dt.date

# TODO: https://likegeeks.com/pandas-to-html-table-styling-css-styler/#Conditional_Formatting - LastSeenAt
#gateway_df['lastSeenAt'] = gateway_df['lastSeenAt'].apply(
#    lambda l: f"{l} &#x2705;" if l == today_str else f"{l} &#x274C;"
#)


gateway_df['placement'] = gateway_df['placement'].apply(
    lambda p: f"{p} &#x1F3E2;" if p == "INDOORS" else f"{p} &#127780;&#65039;"
)

gateway_df = gateway_df.sort_values(by=['gateway_name'],key=lambda col: col.str.lower())

gateway_df['OS2IOT'] = gateway_df.apply(lambda row: f"<a href=\"{os2iot_url}/gateways/gateway-detail/{row['gatewayEUI']}\" target=\"_blank\">{row['gateway_name']}</a>", axis=1)
gateway_df['WMC'] = gateway_df.apply(lambda row: f"<a href=\"https://wmc.wanesy.com/gateways/176/{row['gatewayEUI']}\" target=\"_blank\">{row['gateway_name']}</a>", axis=1)

#gateway_df['phone'] = gateway_df.apply(lambda row: f"<a href=\"sms:{row['phone']}?&body=[admin:pwd4admin] [reboot] system/reboot\" target=\"_blank\">{row['phone']}</a>", axis=1)
gateway_df['phone'] = gateway_df['phone'].apply(
    lambda phone: (
        f'<a href="sms:{phone}?&body=[admin:{kerlink_password}] [reboot] system/reboot" target="_blank">{phone}</a>'
        if isinstance(phone, str) and re.fullmatch(r'\+?\d{10,}', phone)
        else "&#x1F4F5;"
    )
)

''' gateway_df = gateway_df.style.apply(
    lambda col: ['success' if isinstance(d, datetime.date) and d == today_str else 'warning' for d in col],
    subset=['lastSeenAt']
)
'''

html_string = '''
<!doctype html>
<html lang="en">
  <head>
    <title>LoRaWAN Gateways - {today_str}</title>
<!-- Latest compiled and minified CSS -->
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap.min.css" integrity="sha384-HSMxcRTRxnN+Bdg0JdbxYKrThecOKuH5zCYotlSAcp1+c8xmyTe9GYg1l9a69psu" crossorigin="anonymous">
<!-- Optional theme -->
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/css/bootstrap-theme.min.css" integrity="sha384-6pzBo3FDv/PJ8r2KRkGHifhEocL+1X2rVCTTkUfGk7/0pbek5mMa1upzvWbrUbOZ" crossorigin="anonymous">
<!-- Latest compiled and minified JavaScript -->
<script src="https://cdn.jsdelivr.net/npm/bootstrap@3.4.1/dist/js/bootstrap.min.js" integrity="sha384-aJ21OjlMXNL5UyIl/XNwTMqvzeRMZH2w8c5cRVpzpU8Y5bApTppSuUkhZXN0VxHd" crossorigin="anonymous"></script>

<script src="https://cdn.jsdelivr.net/npm/table-sort-js/table-sort.min.js"></script>
  </head>
  <body>
    <div class="container-fluid">
    <h1>LoRaWAN Gateways - {today_str}</h1>
    {table}
    </div>
  </body>
</html>.
'''

with open(os.getcwd() + '/gateways.html', 'w') as f:
    f.write(html_string.format(table=gateway_df.to_html(bold_rows=True,
                                                        index=False, 
                                                        justify='left',
                                                        escape=False, 
                                                        classes='table table-striped table-bordered table-sort table-arrows table-hover'),
                                                        today_str = today_str))
#gateway_df.to_html(os.getcwd() + '/gateways.html', encoding='utf-8', bold_rows=True, index=False, justify='left', escape=False)

In [12]:
gateway_df

,gatewayEUI,gateway_name,phone,placement,lastSeenAt,status,OS2IOT,WMC
0,7276ff002e05024c,2016-LRW-01-Trige-Bibliotek,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
1,7076ff005606039d,2047-LRW-01-Engdalskolen,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
27,7076ff00560908ea,2061-LRW-01-Tranbjergskolen-Groenlykke-Alle,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
51,7276ff002e050230,2065-LRW-01-Bavnehoej-Bibliotek,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
49,7276ff002e050266,2072-LRW-01-Hoejbjerg-Bibliotek,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
...,...,...,...,...,...,...,...,...
80,0016c001f117e482,SenArch-arch0026-Dokk1,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,IN-OPERATION,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/0..."
81,0016c001f117ed6d,SenArch-arch0027-Hoerhaven-offline,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-03-11,OTHER,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/0..."
53,0016c001f1058504,SenArch-Gateway-gs017-Hoerhaven,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2025-05-01,PROJECT,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/0..."
33,7076ff00560907e8,Vikingdalen-offline,&#x1F4F5;,OUTDOORS &#127780;&#65039;,2024-12-13,OTHER,"<a href=""https://prod.os2iot.kmd.dk/gateways/g...","<a href=""https://wmc.wanesy.com/gateways/176/7..."
